# The Battle of Neighborhoods
## --Capstone project

## Introduction the Business Problem

#### I will help Jack Ma look for a new place in HK to open a new cinema.

Jack aims to set up a new cinema in HK to start up his new business in the field of entertainments. I would help him to find a amazing spot for his first cinema.
  
In gerneral, watching movie is part of whole afternoon or night activities. Cinema should have many restaurants and shopping places around it. Transportation is also an important factor. Customer can walk to cinema within few minutess from public transport facilities such as bus stop or subway station.
  
I would focus on the environment of cinema and I have got a list of top 10 favorite cinemas in HK with rating.  

5 potential locations would be pointed out and the best one should be recommended to Jack, the boss.

## Data section

#### 1. geographic coordinate of HK cinemas

First, I will acquire a list of HK cinema with their geographic coordinates from https://hkmovie6.com/cinema .

In [1]:
# Import library
import json
import pandas as pd

In [5]:
# Download cinema list
!wget -O hk_cinema_list.json https://hkmovie6.com/api/cinemas/lists

--2019-05-16 13:58:00--  https://hkmovie6.com/api/cinemas/lists
正在解析主机 hkmovie6.com (hkmovie6.com)... 104.31.66.1, 104.31.67.1
正在连接 hkmovie6.com (hkmovie6.com)|104.31.66.1|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：未指定 [application/json]
正在保存至: “hk_cinema_list.json”

hk_cinema_list.json     [  <=>               ]  52.07K   195KB/s  用时 0.3s    

2019-05-16 13:58:03 (195 KB/s) - “hk_cinema_list.json” 已保存 [53320]



In [6]:
# Convert JSON file to DataFrmae
cinemas_json = None
with open('hk_cinema_list.json', 'r') as f:
    cinemas_json = json.load(f)
    
cinemas = []
for data in cinemas_json['data']:
    cinemas.append({
        'Name': data['name'],
        'Address': data['address'],
        'Latitude': data['lat'],
        'Longitude': data['lon']
    })
df_cinemas = pd.DataFrame(cinemas, columns=['Name','Address','Latitude','Longitude'])

In [8]:
print('There existing {} cinemas in HK for now'.format(len(df_cinemas)))

There existing 70 cinemas in HK for now


In [9]:
df_cinemas.head()

,Name,Address,Latitude,Longitude
0,Emperor Cinemas - Entertainment Building,"3/F, Emperor Cinemas Entertainment Building, 3...",22.281453,114.154230
1,The Coronet @ Emperor Cinemas - Entertainment ...,"3/F, Emperor Cinemas Entertainment Building, 3...",22.281453,114.154230
2,Emperor Cinemas - Tuen Mun,"3/F, New Town Commercial Arcade, 2 Tuen Lee St...",22.390776,113.975983
3,Broadway Circuit - CYBERPORT,"Shop L1 - 3, Level 1, The Arcade, 100 Cyberpor...",22.261067,114.129825
4,Cinema City VICTORIA (Causeway Bay),"2-8 Sugar Street, Causeway Bay, Hong Kong",22.279805,114.187126


The geographic coordinates of 5 potential cinemas.

In [22]:
p_locations = [
    { 'Location': 'L1', 'Address': 'Sau Mau Ping Shopping Centre, Sau Mau Ping', 'Latitude': 22.319503, 'Longitude' : 114.232187},
    { 'Location': 'L2', 'Address': 'Tuen Mun Ferry, Tuen Mun', 'Latitude': 22.371780, 'Longitude' : 113.966039},
    { 'Location': 'L3', 'Address': 'Un Chau Shopping Centre, Cheung Sha Wan', 'Latitude': 22.337280, 'Longitude' : 114.156457},
    { 'Location': 'L4', 'Address': 'Prosperity Millennia Plaza, North Point', 'Latitude': 22.291698, 'Longitude' : 114.208168},
    { 'Location': 'L5', 'Address': 'Tsuen Fung Centre Shopping Arcade, Tsuen Wan', 'Latitude': 22.372112, 'Longitude' : 114.119317},
]

Dataframe of 5 potential locations with geographic coordinates information

In [24]:
df_p_locations = pd.DataFrame(p_locations, columns=['Location', 'Address', 'Latitude', 'Longitude'])
df_p_locations

,Location,Address,Latitude,Longitude
0,L1,"Sau Mau Ping Shopping Centre, Sau Mau Ping",22.319503,114.232187
1,L2,"Tuen Mun Ferry, Tuen Mun",22.371780,113.966039
2,L3,"Un Chau Shopping Centre, Cheung Sha Wan",22.337280,114.156457
3,L4,"Prosperity Millennia Plaza, North Point",22.291698,114.208168
4,L5,"Tsuen Fung Centre Shopping Arcade, Tsuen Wan",22.372112,114.119317


#### 4. Favorite cinema list from Jack

The favorite cinema list provide information that I could use to choose the best location and the rating is range of 1.0 (worst) to 5.0 (best) values.

In [27]:
Jack_list = [
    {'Name': 'Boradway Circuit - MONGKONG', 'Rating': 4.5},
    {'Name': 'Boradway Circuit - The ONE', 'Rating': 4.5},
    {'Name': 'Grand Ocean', 'Rating': 4.3},
    {'Name': 'The Grand Cinema', 'Rating': 3.4},
    {'Name': 'AMC Pacific Place', 'Rating': 2.3},
    {'Name': 'UA IMAX @ Airport', 'Rating': 1.5},
]

df_Jack_list = pd.DataFrame(Jack_list, columns=['Name','Rating'])

df_Jack_list

,Name,Rating
0,Boradway Circuit - MONGKONG,4.5
1,Boradway Circuit - The ONE,4.5
2,Grand Ocean,4.3
3,The Grand Cinema,3.4
4,AMC Pacific Place,2.3
5,UA IMAX @ Airport,1.5


#### 4. Surroundings
The recommended cinema location needs to have many eating and shopping venues nearby. Convenient public transport is a big plus. FourSquare API would be used to find these venues and 5 minutes walking distance is about 500m. This should be suitable distance to search nearby venues.

Due to the API limitation, it would be better to search venues by category. Following categories will be used for finding the target venues. Full list of categories: https://developer.foursquare.com/docs/resources/categories

In [28]:
cinema = df_cinemas.loc[0]

In [29]:
print('Use the first cinema "{}" in the list as example to explore venues nearyby'.format(cinema['Name']))

Use the first cinema "Emperor Cinemas - Entertainment Building" in the list as example to explore venues nearyby


In [30]:
fs_categories = {
    'Food': '4d4b7105d754a06374d81259',
    'Shop & Service': '4d4b7105d754a06378d81259',
    'Bus Stop': '52f2ab2ebcbc57f1066b8b4f',
    'Metro Station': '4bf58dd8d48988d1fd931735',
    'Nightlife Spot': '4d4b7105d754a06376d81259',
    'Arts & Entertainment': '4d4b7104d754a06370d81259'
}

In [31]:
# Install FourSquare client library
!pip install foursquare

  Stored in directory: /Users/apple/Library/Caches/pip/wheels/50/82/db/97c078881efb89f4e89560487926fbadfff87a40e5fe41f6fb
Successfully built foursquare
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [48]:
import foursquare
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests
fs = foursquare.Foursquare(client_id= 'CLIENT_ID', client_secret= 'CLIENT_SECRET')
RADIUS = 500


In [52]:
# Define a function to search nearby information and convert the result as dataframe
def venues_nearby(latitude, longitude, category):    
    results = fs.venues.search(
        params = {
            'query': category, 
            'll': '{},{}'.format(latitude, longitude),
            'radius': RADIUS,
            'categoryId': fs_categories[category]
        }
    )    
    df = json_normalize(results['venues'])
    cols = ['Name','Latitude','Longitude']    
    if( len(df) == 0 ):        
        df = pd.DataFrame(columns=cols)
    else:        
        df = df[['name','location.lat','location.lng']]
        df.columns = cols
    print('{} "{}" venues are found within {}m of location'.format(len(df), category, RADIUS))
    return df
    

In [53]:
venues_nearby(cinema['Latitude'], cinema['Longitude'], 'Metro Station').head()

2 "Metro Station" venues are found within 500m of location


,Name,Latitude,Longitude
0,MTR Central Station (港鐵中環站),22.282115,114.158221
1,MTR Hong Kong Station (港鐵香港站),22.285054,114.158379


In [54]:
venues_nearby(cinema['Latitude'], cinema['Longitude'], 'Bus Stop').head()

30 "Bus Stop" venues are found within 500m of location


,Name,Latitude,Longitude
0,Seymour Road / Robinson Road Bus Stop 西摩道／羅便臣道巴士站,22.280465,114.150347
1,HSBC Main Building / Queen's Road Central Bus ...,22.280329,114.159638
2,Hang Seng Bank Head Office Bus Stop 恒生銀行總行巴士站,22.283998,114.156038
3,Douglas Street Bus Stop 德忌利士街巴士站,22.283131,114.157010
4,Lan Kwai Fong / D'Aguilar Street Bus Stop 蘭桂坊／...,22.281437,114.156131


In [55]:
venues_nearby(cinema['Latitude'], cinema['Longitude'], 'Food').head()

26 "Food" venues are found within 500m of location


,Name,Latitude,Longitude
0,Mana! Fast Slow Food,22.282921,114.154651
1,Good Luck Thai Food (鴻運泰國美食),22.281165,114.155296
2,Soul Food,22.281668,114.152495
3,nood food,22.283088,114.155551
4,Chiu Lung Fast Food (昭隆美食),22.282659,114.156753


In [56]:
venues_nearby(cinema['Latitude'], cinema['Longitude'], 'Arts & Entertainment').head()

12 "Arts & Entertainment" venues are found within 500m of location


,Name,Latitude,Longitude
0,Tai Kwun Centre for Heritage and Arts (大館古蹟及藝術館),22.281224,114.154032
1,Wah Tung China Arts Limited (華通陶瓷藝術有限公司),22.283046,114.152723
2,Ravenel Fine Arts Limited 睿芙奧,22.281819,114.156906
3,KONG Arts Space,22.281751,114.153300
4,State Of The Arts,22.282225,114.155006


Furthermore, a content-based recommend system could be setup to resolve the problem.  

Combine with FourSquare API on counting the numbers of different venues (Food, Transport, Night Life) and Hong Kong cinema list, a cinema nearby venues matrix can be built. Jack's favorite list is sample to combine with cinema nearby venues matrix to become a weighted matrix of favorite cinemas.

The weighted matrix can be applied to generate a ranking result. The top in the ranking list can be recommended to Jack.


## Methodology 

## Results 

## Discussion 

## Conclusion 